In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
#input_size = 784
#hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# 1 Этап - подготовка данных!

In [2]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


# 2 Этап - Нейронная сеть!

In [3]:
# Fully connected neural network with one hidden layer
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(#input batch_size*N*N*1
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) #batch_size*N/2*N/2*16
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) #batch_size*N/4*N/4*32
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        #input batch_size*N*N*1
        out = self.layer1(x) #batch_size*N/2*N/2*16
        out = self.layer2(out)#batch_size*N/4*N/4*32
        out = out.reshape(out.size(0), -1)#batch_size*(N/4*N/4*32)
        out = self.fc(out)#batch_size*num_classes
        return out

model = ConvNet(num_classes).to(device)


# 3 этап - Loss-функция и оптимизатор

In [4]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 4 этап - обучение нейросети

In [6]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

    # Save the model checkpoint
    torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.1636
Epoch [1/5], Step [200/600], Loss: 0.0856
Epoch [1/5], Step [300/600], Loss: 0.0829
Epoch [1/5], Step [400/600], Loss: 0.0611
Epoch [1/5], Step [500/600], Loss: 0.0845
Epoch [1/5], Step [600/600], Loss: 0.0920
Accuracy of the network on the 10000 test images: 98.36 %
Epoch [2/5], Step [100/600], Loss: 0.0306
Epoch [2/5], Step [200/600], Loss: 0.0662
Epoch [2/5], Step [300/600], Loss: 0.0087
Epoch [2/5], Step [400/600], Loss: 0.0086
Epoch [2/5], Step [500/600], Loss: 0.0749
Epoch [2/5], Step [600/600], Loss: 0.0362
Accuracy of the network on the 10000 test images: 98.72 %
Epoch [3/5], Step [100/600], Loss: 0.0197
Epoch [3/5], Step [200/600], Loss: 0.0401
Epoch [3/5], Step [300/600], Loss: 0.0470
Epoch [3/5], Step [400/600], Loss: 0.0978
Epoch [3/5], Step [500/600], Loss: 0.1071
Epoch [3/5], Step [600/600], Loss: 0.0234
Accuracy of the network on the 10000 test images: 99.11 %
Epoch [4/5], Step [100/600], Loss: 0.0093
Epoch [4/5], Step [200/600],

In [14]:
ss = []
for p in model.parameters():
    ss.append(p.numel())
print(ss)
sum(ss)

[400, 16, 16, 16, 12800, 32, 32, 32, 15680, 10]


29034